In [ ]:
from datetime import datetime, timedelta
import numpy as np
import pandas as pd
import aiohttp
import matplotlib.pyplot as plt
import networkx as nx
from gnn_package import preprocessing

adj_matrix_dense, node_ids, metadata = preprocessing.load_graph_data(
    prefix="25022025_test", return_df=False
)
name_id_map = preprocessing.get_sensor_name_id_map()
# node_names = [name_id_map[str(node_id)] for node_id in node_ids]
adj_matrix_dense.max()

In [ ]:
adj = preprocessing.compute_adjacency_matrix(adj_matrix_dense)
adj[0].max()

In [ ]:
from private_uoapi import (
    LightsailWrapper,
    LSAuth,
    LSConfig,
    DateRangeParams,
    convert_to_dataframe,
)

config = LSConfig()
auth = LSAuth(config)
client = LightsailWrapper(config, auth)

print(f"Using base URL: {config.base_url}")
print(f"Using username: {config.username}")
print(f"Using secret key: {'*' * len(config.secret_key)}")  # Mask the secret key

In [ ]:
sensor_locations = client.get_traffic_sensors()
sensor_locations = pd.DataFrame(sensor_locations)
display(sensor_locations.head())

date_range_params = DateRangeParams(
    start_date=datetime(2024, 2, 18, 0, 0, 0),
    end_date=datetime(2024, 2, 18, 7, 59, 59),  # Just under 8 hours so we get 1 record
    max_date_range=timedelta(days=10),
)

count_data = await client.get_traffic_data(date_range_params)

In [ ]:
counts_df = convert_to_dataframe(count_data)

In [ ]:
len(counts_df)

In [ ]:
counts_dict = {}
for location in sensor_locations["location"]:
    df = counts_df[counts_df["location"] == location]
    series = pd.Series(df["value"].values, index=df["dt"])
    location_id = name_id_map[location]
    counts_dict[location_id] = series if not df.empty else None

len([series for series in counts_dict.values() if series is not None])

In [ ]:
def plot_graph(adj):
    plt.figure(figsize=(10, 5))
    rows, cols = np.where(adj > 0)
    edges = zip(rows.tolist(), cols.tolist())
    G = nx.Graph()
    G.add_edges_from(edges)
    nx.draw(G, with_labels=True)
    plt.show()

In [ ]:
results_containing_data = {
    node_id: data for node_id, data in counts_dict.items() if data is not None
}

In [ ]:
len(results_containing_data)

In [ ]:
counts_dict["10000"]

In [ ]:
counts_dict["10000"].dtypes

In [ ]:
processor = preprocessing.TimeSeriesPreprocessor(12, 1, pd.Timedelta(minutes=15), -1)

In [ ]:
windows = processor.create_windows(results_containing_data)

In [ ]:
print(f"Windows type: {type(windows)}")
print(f"Windows length: {len(windows)}")
print(f"Windows tuple element type: {type(windows[0])}")
print(f"Windows tuple length: {len(windows[0])}")
print(f"Shape of sensor data for sample window: {windows[0]['10005'].shape}")
print(f"Shape of missing data mask for sample window: {windows[1]['10005'].shape}")
print(f"Length of metadata for sample window: {len(windows[2]['10005'])}")

In [ ]:
len(results_containing_data)

In [ ]:
len(windows[0])

In [ ]:
windows[0].keys()

In [ ]:
for sensor_name in windows[0].keys():
    print(f"{sensor_name}: {windows[0][sensor_name].shape})")